In [1]:
import Levenshtein
import pandas as pd
import numpy as np
import re

In [2]:
str1='2017年8月28日9时45分, 李宏印驾驶的冀C79805冀CR689挂号货物运输车辆，行驶至110高速公路综合检查站，经执法人员武翼（执法证号：11310165）、吴建勋（执法证号：11310123）示证检查：该车取得普通货物道路运输证，拉煤，从辽宁到河北，该车驾驶员无《从业资格证》。当事人李宏印的行为构成了不符合规定条件的人员驾驶道路运输经营车辆。'
str2='2017年4月20日12时30分, 梁北红驾驶的京QQ99J8号货物运输车辆，行驶至延庆区110高速公路综合检查站，经执法人员张伟（执法证号：11310095）、耿文东（执法证号：11310167）示证检查：该车取得普通货物道路运输证，拉汽车，从内蒙古到北京，该车驾驶员无《从业资格证》。当事人梁北红的行为构成了不符合规定条件的人员驾驶道路运输经营车辆。'
str3='2017年9月份，在未取得林业主管部门批准的情况下，擅自将自家位于本村二道沟（小地名）退耕还林地内的8株杨树砍伐。经鉴定，砍伐的8株杨树总蓄积量为2.358立方米，价值为人民币459.81元。'

In [3]:
sim = Levenshtein.distance(str1, str3)
sim

164

In [4]:
sim = Levenshtein.ratio(str1, str3)
sim

0.11678832116788321

In [5]:
file = open("D:\\NLP\\case_recommender\\recommender.txt", 'a', encoding='utf-8')
basic_df = pd.read_csv('D:\\NLP\\case_recommender\\data\\basic_bj.csv')

In [6]:
stopwords = set()
with open('D:\\NLP\\case_recommender\\data\\stopwords.txt', "r", encoding="utf-8") as f_stopwords:
    for line in f_stopwords:
        stopwords.add(line.strip())

In [7]:
x_test=list()
case_id_list=list()
case_order_list=list()
pattern = re.compile('\s+')
with open('D:\\NLP\\case_recommender\\data\\test2.txt','r',encoding='utf-8') as test_file:
    for line in test_file.readlines():
#         print(basic_df[basic_df.ID == line.split('\t')[0]].index.tolist()[0])
#         break
        case_id_list.append(line.split('\t')[0])
        case_order_list.append(basic_df[basic_df.ID == line.split('\t')[0]].index.tolist()[0])
        line=line.split('\t')[2]
        dataline = [x for x in re.sub(pattern, '', line) if x not in stopwords]
        dataline=''.join(dataline)
        x_test.append(dataline)

In [19]:
pd.set_option("display.max_colwidth",2000)
target = np.random.randint(0, high=len(x_test))
c = case_id_list[target]
target_line1 = basic_df[basic_df.ID == c]
targic_order=basic_df[basic_df.ID == c].index.tolist()[0]
target_line=re.sub(pattern, '', basic_df[basic_df.ID == c].ILLEGAL_FACTS.to_string(index=False))
target_line

'2017年6月5日11时10分,李强驾驶的辽H40172辽HL955挂号货物运输车辆，行驶至延庆区110高速公路综合检查站，经执法人员武翼（执法证号：11310165）、吴建勋（执法证号：11310123）示证检查：该车取得普通货物道路运输证，拉有机肥，从甘肃到辽宁，该车驾驶员无《从业资格证》。当事人李强的行为构成了不符合规定条件的人员驾驶道路运输经营车辆。'

In [20]:
r_file=open ('recommender.txt','a',encoding='utf-8')
r_file.write("####################### 待推荐案例 ##########################\n")
r_file.write(target_line+'\n\n')
r_file.write("####################### 推荐结果 ############################\n")
# r_file.close()

58

In [21]:
recommander=dict()
#开始推荐
for case_order in case_order_list:
    candidate_line=basic_df.iloc[case_order].ILLEGAL_FACTS
    sim=Levenshtein.jaro_winkler(target_line, candidate_line)
    recommander[case_order] = sim


In [22]:
print("####################### 待推荐案例 ##########################\n")
print(target_line )
print("####################### 推荐结果 ############################\n")
for pre in sorted(recommander.items(),key=lambda x:x[1],reverse=True)[1:11]:
#     basic_df[basic_df.ID ==pre[0]].ILLEGAL_FACTS.to_csv('recommender_transport.txt', mode='a', header=False)
    r_file.write(re.sub(pattern, '',basic_df.iloc[pre[0]].ILLEGAL_FACTS)+'\n\n')
    print(basic_df.iloc[pre[0]].ILLEGAL_FACTS)
r_file.write('\n')
r_file.close()

####################### 待推荐案例 ##########################

2017年6月5日11时10分,李强驾驶的辽H40172辽HL955挂号货物运输车辆，行驶至延庆区110高速公路综合检查站，经执法人员武翼（执法证号：11310165）、吴建勋（执法证号：11310123）示证检查：该车取得普通货物道路运输证，拉有机肥，从甘肃到辽宁，该车驾驶员无《从业资格证》。当事人李强的行为构成了不符合规定条件的人员驾驶道路运输经营车辆。
####################### 推荐结果 ############################

2017年6月19日20时55分, 聂飞飞驾驶的冀G84070冀GW065挂号货物运输车辆，行驶至110高速公路综合检查站，经执法人员武翼（执法证号：11310165）、吴建勋（执法证号：11310123）示证检查：该车取得普通货物道路运输证，拉煤，从河北到天津，该车驾驶员无《从业资格证》。当事人聂飞飞 的行为构成了不符合规定条件的人员驾驶道路运输经营车辆。
2017年6月2日11时10分, 高海龙驾驶的辽P71415辽PK025挂号货物运输车辆，行驶至延庆区110高速公路综合检查站，经执法人员武翼（执法证号：11310165）、吴建勋（执法证号：11310123）示证检查：该车取得普通货物道路运输证，拉焦炭，从宁夏到辽宁，该车驾驶员无《从业资格证》。当事人高海龙 的行为构成了不符合规定条件的人员驾驶道路运输经营车辆。
2017年6月11日11时10分, 李春涛驾驶的辽PD0770辽P2622挂号货物运输车辆，行驶至110高速公路综合检查站，经执法人员武翼（执法证号：11310165）、吴建勋（执法证号：11310123）示证检查：该车取得普通货物道路运输证，拉煤，从山西到辽宁，该车驾驶员无《从业资格证》。当事人李春涛 的行为构成了不符合规定条件的人员驾驶道路运输经营车辆。
2017年6月11日10时30分, 杨君驾驶的辽956520辽P2296挂号货物运输车辆，行驶至110高速公路综合检查站，经执法人员武翼（执法证号：11310165）、吴建勋（执法证号：11310123）示证检查：该车取得普通货物道路运输证，拉煤，从山西到辽宁，该车驾驶员无《从业资格证》。当事人杨君 的行为构成

In [7]:
Levenshtein.jaro_winkler('william', 'willlaim')

0.9464285714285714